In [1]:
import sqlite3
import pandas as pd
import operator
from typing import Annotated, TypedDict
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langgraph.types import Send
from langgraph.graph import StateGraph, START, END
import threading
from datetime import datetime, timedelta
import numpy as np

In [2]:
def create_database():
    conn = sqlite3.connect("inventory_management.db")
    cursor = conn.cursor()
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS warehouses (
            warehouse_id INTEGER PRIMARY KEY,
            name TEXT,
            location TEXT,
            capacity INTEGER
    )""")
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS products (
            product_id INTEGER PRIMARY KEY,
            name TEXT,
            category TEXT,
            unit_price DECIMAL(10,2),
            reorder_point INTEGER
        )""")
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS inventory (
            warehouse_id INEGER,
            product_id INTEGER,
            current_stock INTEGER,
            last_updated DATE,
            FOREIGN KEY (warehouse_id) REFERENCES warehouses(warehouse_id),
            FOREIGN KEY (product_id) REFERENCES products(product_id)
        )""")
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS sales (
            sale_id INTEGER PRIMARY KEY,
            warehouse_id INTEGER,
            product_id INTEGER,
            sale_date DATE,
            quantity INTEGER,
            FOREIGN KEY (warehouse_id) REFERENCES warehouses(warehouse_id),
            FOREIGN KEY (product_id) REFERENCES products(product_id)
        )""")
    
    conn.commit()
    conn.close()

create_database()

In [3]:
def insert_sample_data():
    conn = sqlite3.connect("inventory_management.db")
    cursor = conn.cursor()
    
    warehouses = [
        (1, 'North Hub', 'Seattle', 10000),
        (2, 'Central Distribution', 'Dallas', 15000),
        (3, 'East Coast Center', 'Atlanta', 12000)
    ]
    cursor.executemany('INSERT OR REPLACE INTO warehouses VALUES(?,?,?,?)', warehouses)
    
    products = [
        (1, 'Laptop Pro', 'Electronics', 999.99, 50),
        (2, 'Smartphone X', 'Electronics', 699.99, 75),
        (3, 'Wireless Earbuds', 'Electronics', 129.99, 100),
        (4, 'Smart Watch', 'Electronics', 299.99, 60),
        (5, 'Tablet Ultra', 'Electronics', 449.99, 40)
    ]
    cursor.executemany('INSERT OR REPLACE INTO products VALUES(?,?,?,?,?)', products)
    
    inventory = [
        (1, 1, 45, '2024-03-15'),
        (1, 2, 80, '2024-03-15'),
        (1, 3, 95, '2024-03-15'),
        (2, 1, 55, '2024-03-15'),
        (2, 2, 65, '2024-03-15'),
        (2, 4, 58, '2024-03-15'),
        (3, 3, 85, '2024-03-15'),
        (3, 4, 52, '2024-03-15'),
        (3, 5, 35, '2024-03-15')
    ]
    cursor.executemany('INSERT OR REPLACE INTO inventory VALUES(?,?,?,?)', inventory)
    
    sales = []
    for day in range(30):
        date = (datetime.now() - timedelta(days=day)).strftime("%Y-%m-%d")
        for _ in range(5):
            warehouse_id = np.random.randint(1, 4)
            product_id = np.random.randint(1, 6)
            quantity = np.random.randint(1, 10)
            sales.append((None, warehouse_id, product_id, date, quantity))
    cursor.executemany('INSERT INTO sales VALUES(?,?,?,?,?)', sales)
    
    conn.commit()
    conn.close()
    
insert_sample_data()